В данном ноутбуке я смотрю на отзывы, сгерерированные моделью gpt-2 "117M" (это предтренированная модель с 117 миллионами параметров, которая дообучается на нашем датасете).

! Данный ноутбук нужно запускать из Google Colab !
Пометка для себя - я это делал под аккаунтом garnov.yud@phystech.edu

Импортируем нужные модули, gpt-2 работает с tensorflow первой версии.

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Проверяем, что сейчас мы используем GPU.

In [0]:
import numpy as np 
import pandas as pd 
import bz2
import gc
import chardet
import re
import os
import tensorflow as tf

delim = '==============================================='

Чтобы не потерять прогресс, добавим Google Drive как хранилище, чтобы gpt2 могло складывать туда результаты и модели.

In [3]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Загружаем модель. Я использовал самую малую "117М", потому что обучение на других занимает очень много времени и Google Colab нередко падает из-за нехватки RAM. Для более точных результатов советую взять версию побольше (см. тут https://github.com/openai/gpt-2/issues/209)

In [0]:
# model_name = "117M"
# if not os.path.isdir(os.path.join("models", model_name)):
# 	print(f"Downloading {model_name} model...")
# 	# gpt2.download_gpt2(model_dir=root_path + '/models', model_name=model_name)   # model is saved into current directory under /models/774M/
# 	gpt2.download_gpt2(model_name=model_name)   # model is saved into current directory under /models/774M/

Начинаем сессию

In [0]:
gc.collect()
gpt2.copy_checkpoint_from_gdrive()


In [6]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


Попробуем поиграться с разными параметрами генерирования текстов, например длиной.

In [7]:
for length in [le for le in range(10, 65, 10)]:
  print(delim + str(length) + delim)
  gpt2.generate(sess, length=length)
  print('\n')

===============================================10===============================================
great movie for the price so don't be fooled


===============================================20===============================================
"i am so sorry" lyrics - even my favorite song is still right there. this album is


===============================================30===============================================
i was really disappointed in this movie. the movie was a waste of time and i was forced to watch it over and over. i felt like i


===============================================40===============================================
"a little boy is about to be born". it is a boy named julie who is very happy and happy, but when he is about to be born, he is sent to a school


===============================================50===============================================
if you are looking for an easy way to set up an internet router, broadband, or any other router for use o

Как мы видим для каждой длины модель генерирует человекоподобные отзывы. Однако есть ощущение, что они какие-то незаконченные. Можем дать модели возможность самой определять когда ей нужно остановиться и обрезать отзывы по \n. Так же можно задать с чего модели начать отзыв, то есть можно генерировать отзывы о конкретной вещи. Давайте попробуем дать ей тему для отзыва.

In [16]:
topics = ['The book', 'This film', 'The album', 'The phone']

for topic in topics:
    print(delim + str(topic) + delim)
    gpt2.generate(sess, prefix=topic, truncate='\n', length=50)
    print('\n')

===============================================The book===============================================
The book was enjoyable but didn't keep me reading. i hope you enjoy it.


===============================================This film===============================================
This film is very, very well made. it is an intriguing story, but it takes a while to get to the point. i would not recommend this to anyone.


===============================================The album===============================================
The album basically is a collection of songs that were released in the 0000s, and they have been given a new life over the years. the songwriting is just plain cool, and the lyrics are all very catchy. the only problem is that the songwriting


===============================================The phone===============================================
The phone is the perfect device for a home phone. it has a good volume. i have a full-size phone and the older one is 00/0

Удивительно, но модель даже довольно хорошо справляется с названиями продуктов, не только с конкретной категорией.

In [22]:
name = 'Samsung Galaxy S8 '
gpt2.generate(sess, prefix=name, truncate='\n', length=80, include_prefix=True)

Samsung Galaxy S8 is iced, by far the worst. to get it you have to buy it from a website or buy the product from a store. it has a video buffering problem, very annoying, in my opinion. the price is right, and the customer service is a mess. i will not buy this again.


Однако если дать модели название товара абсолютно такое же как и в Amazon, то модель будет удивлена и не поймёт что мы от неё хотим, так как названия у товаров обычно содержат очень много технических характеристик, которые не особо много информации дают для отзыва. Точнее это слишком сложный контекст для модели.

In [23]:
sg_tab = 'Samsung Galaxy Tab A 10.1 Inch (T510) 32 GB WiFi Tablet Silver (2019), Silver'
gpt2.generate(sess, prefix=sg_tab, truncate='\n', length=50, include_prefix=True)

Samsung Galaxy Tab A 10.1 Inch (T510) 32 GB WiFi Tablet Silver (2019), Silver (0000)


О параметре top_k сами разработчики отзываются следующим образом: 

`
top_k=0: Integer value controlling diversity. 1 means only 1 word is considered for each step (token), resulting in deterministic completions, while 40 means 40 words are considered at each step. 0 is a special setting meaning no restrictions. 40 generally is a good value.
`

In [31]:
def generate_samples(session, kwarg_name, kwarg_list: list, **kwargs):
    for kwarg in kwarg_list:
        print(delim + str(kwarg) + delim)
        gpt2.generate(session, length=50, **{kwarg_name: kwarg}, truncate='\n', **kwargs)
        print('\n')

ks = [20, 40, 60]
generate_samples(sess, 'top_k', ks)

===============================================20===============================================
"a man who cannot live" is the book that will forever define the genre of the 00th century. the authors have made one hell of a world of literature. they are a great author, but they are also a great writer. they should


===============================================40===============================================
great fun, good read, it's a great book!


===============================================60===============================================
as the other reviewer stated, the black box was not included in the purchase price. it was too small and the box was too small. i was able to keep it for 0 hours and finally received the box. it is not worth the price.




Параметр temperature означает следующее:

`:temperature=1: Float value controlling randomness in boltzmann distribution. Lower temperature results in less random completions. As the temperature approaches zero, the model will become deterministic and repetitive. Higher temperature results in more random completions.`

То есть, для отзывов непохожих на отзывы из датасета нужно выбрать температуру как можно ближе к 1. Для похожих наоборот.

In [32]:
temps = [0.1, 0.4, 1.0]
generate_samples(sess, 'temperature', temps, top_k=40)

===============================================0.1===============================================
"the best of the best" is a great book. i read it in the middle of the night, and i was hooked. i have read it many times, and i still love it. i have read it many times, and i still


===============================================0.4===============================================
"the world of the kiddies" is a very good movie, but it's not a "movie" either. it's a "movie" with a lot of action, and a lot of nudity. it's not a "movie"


===============================================1.0===============================================
toilet humor with a flair for the outrageous. don't waste your money!




Заключение: gpt-2 несмотря на то, что я обучил модель не полном датасете и взял самую маленькую версию модели из-за нехватки ресурсов, прекрасно справляется с генерированием различного рода текстов, в частности отзывов. Отзывы выглядят очень человечными и складными. Кроме того, данную модель можно ещё несколько раз дообучать на выборке, чтобы снизить loss, что ещё улучшит качество генерирования текстов.